In [ ]:
import zipfile

zip_path = '/content/drive/MyDrive/Data Mining/DAMING FIXX/dataset gabungan.zip'
extract_path = '/content'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
import time

# Parameter aplikasi
IMAGE_SIZE = 224
CLASSES = ['Black Rot', 'ESCA', 'Healthy', 'Leaf Blight']

@st.cache_resource
def load_model():
    try:
        model = tf.keras.models.load_model("/content/drive/MyDrive/Data Mining/DAMING FIXX/saved_model_potato_CNN.keras")
        return model
    except Exception as e:
        st.error(f"Terjadi kesalahan saat memuat model: {e}")
        return None

model = load_model()

# Fungsi prediksi
def predict(image):
    try:
        image_array = img_to_array(image)
        image_array = tf.image.resize(image_array, (IMAGE_SIZE, IMAGE_SIZE)) / 255.0
        image_array = np.expand_dims(image_array, axis=0)
        prediction = model.predict(image_array)
        predicted_class = CLASSES[np.argmax(prediction)]
        confidence = round(100 * np.max(prediction), 2)
        return predicted_class, confidence
    except Exception as e:
        st.error(f"Terjadi kesalahan saat melakukan prediksi: {e}")
        return None, None

# Fungsi untuk menampilkan EDA
def eda(directory):
    try:
        st.subheader("Exploratory Data Analysis (EDA)")
        st.write("Informasi Dataset:")

        # Hitung jumlah gambar per kelas
        data_info = {}
        for folder in os.listdir(directory):
            folder_path = os.path.join(directory, folder)
            if os.path.isdir(folder_path):
                data_info[folder] = len(os.listdir(folder_path))

        if data_info:
            # Tampilkan tabel
            st.table({"Kelas": list(data_info.keys()), "Jumlah Gambar": list(data_info.values())})

            # Visualisasi jumlah gambar per kelas
            fig, ax = plt.subplots()
            ax.bar(data_info.keys(), data_info.values(), color=["blue", "green", "orange", "red"])
            ax.set_title("Distribusi Gambar per Kelas")
            ax.set_xlabel("Kelas")
            ax.set_ylabel("Jumlah Gambar")
            st.pyplot(fig)
        else:
            st.error("Folder dataset kosong atau tidak valid.")
    except Exception as e:
        st.error(f"Terjadi kesalahan saat melakukan EDA: {e}")

# UI Deployment Streamlit
def main():
    st.markdown(
        """
        <style>
        body {
            background: linear-gradient(to right, #56ccf2, #2f80ed);
            font-family: 'Arial', sans-serif;
        }
        .title {
            font-size: 42px;
            color: white;
            font-weight: bold;
        }
        .subheader {
            color: white;
            font-size: 24px;
            margin-bottom: 20px;
        }
        .upload-area {
            border: 2px dashed #fff;
            padding: 20px;
            border-radius: 10px;
            text-align: center;
            color: white;
        }
        .btn-predict {
            background-color: #56ccf2;
            color: white;
            font-size: 18px;
            padding: 10px 20px;
            border: none;
            border-radius: 5px;
            transition: 0.5s;
        }
        .btn-predict:hover {
            background-color: #2f80ed;
            cursor: pointer;
        }
        .footer {
            text-align: center;
            font-size: 14px;
            color: white;
        }
        </style>
        """,
        unsafe_allow_html=True,
    )

    # Menu utama
    menu = ["Prediksi", "EDA"]
    choice = st.sidebar.selectbox("🌿 Menu Utama", menu)

    # Judul halaman
    st.markdown("<div class='title'>Prediksi Penyakit Daun Anggur 🍇</div>", unsafe_allow_html=True)

    # Deskripsi dinamis berdasarkan menu
    if choice == "Prediksi":
        st.write(
            "Website ini dirancang untuk memprediksi penyakit daun anggur. "
            "Unggah gambar penyakit daun anggur di bawah ini dan klik 'Prediksi' untuk mengetahui hasilnya."
        )
        st.markdown("<div class='subheader'>Prediksi Penyakit Daun</div>", unsafe_allow_html=True)
        st.markdown("<div class='upload-area'>Unggah gambar daun anggur (.jpg, .jpeg, .png)</div>", unsafe_allow_html=True)

        # Fungsi unggah dan prediksi gambar
        uploaded_file = st.file_uploader("", type=["jpg", "jpeg", "png"])

        if uploaded_file is not None:
            try:
                image = Image.open(uploaded_file)
                st.image(image, caption="Gambar yang diunggah", use_column_width=True)

                if st.button("🔍 Prediksi", key="predict", use_container_width=True):
                    with st.spinner('🔄 Memprediksi gambar...'):
                        time.sleep(2)
                        predicted_class, confidence = predict(image)

                    if predicted_class:
                        st.success(f"✅ Kelas Prediksi: {predicted_class}")
                        st.info(f"📊 Tingkat Kepercayaan: {confidence}%")
                    else:
                        st.error("❌ Gagal melakukan prediksi.")
            except Exception as e:
                st.error(f"⚠️ Terjadi kesalahan saat memprediksi gambar: {e}")

    elif choice == "EDA":
        st.write(
            "Exploratory Data Analysis (EDA) digunakan untuk mengeksplorasi data pada dataset. "
            "Dengan EDA, kita dapat mengetahui distribusi data, jumlah kelas, dan pola lain yang ada pada dataset."
        )
        st.markdown("<div class='subheader'>Exploratory Data Analysis (EDA)</div>", unsafe_allow_html=True)
        dataset_path = st.text_input("📂 Masukkan path dataset", "/content/dataset sekunder+primer/train")

        if dataset_path:
            if os.path.exists(dataset_path):
                eda(dataset_path)
            else:
                st.error("❌ Path dataset tidak valid atau tidak ditemukan.")
        else:
            st.info("ℹ️ Masukkan path dataset untuk melakukan analisis.")

    # Footer
    st.markdown(
    """
    ---
    <div class="footer">
        <p>Website ini dirancang dan dibangun oleh <strong>Kelompok 1</strong> menggunakan <a href="https://streamlit.io" target="_blank" style="color: #56ccf2; text-decoration: none;">Streamlit</a> ❤️</p>
        <p>Terima kasih telah menggunakan aplikasi kami untuk memprediksi penyakit daun anggur! 😊</p>
    </div>
    """,
    unsafe_allow_html=True,
    )


if __name__ == "__main__":
    main()

Overwriting app.py


In [ ]:
# Menginstall Streamlit untuk deployment website
!pip install streamlit -q

# Mendapatkan alamat IP Publik
!wget -q -O - ipv4.icanhazip.com

35.227.69.216


In [ ]:
# Menginstall localtunnel untuk menjalankan website
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
changed 22 packages in 4s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏

In [ ]:
# Menjalankan app.py
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.227.69.216:8501

⠼⠴your url is: https://tired-trains-stop.loca.lt
2024-12-17 05:38:24.385537: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-17 05:38:24.415300: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-17 05:38:24.424157: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-17 05:38:25.926909: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-12-17 05